In [1]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, JSON
load_dotenv()

True

## Semantic search

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("resources/acmecorp-employee-handbook.pdf")

data = loader.load()

print(data)

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-11-20T23:23:16+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-11-20T23:23:16+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resources/acmecorp-employee-handbook.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Employee Handbook\nNon-Disclosure Agreement (NDA) Policy\nEmployees must protect confidential information belonging to the company, its clients, and partners.\nThis includes, but is not limited to, product roadmaps, customer data, internal communications,\nproprietary algorithms, financial information, and unreleased features. Confidential information may not\nbe shared with unauthorized individuals inside or outside the organization. These obligations continue\nafter employment ends.\nWorkplace Conduct Policy\nEmployees must maintain a respectful, professional environment

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

3


Embedding Models: https://docs.langchain.com/oss/python/integrations/text_embedding

In [4]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# AWS Titan Embeddings V2
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    region_name="us-east-1"
)

In [5]:
# from langchain_core.vectorstores import InMemoryVectorStore
# vector_store = InMemoryVectorStore(embeddings)

# Pinecone Vector Store
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os

# Verificar que la API key existe
api_key = os.getenv("PINECONE_API_KEY")
if not api_key:
    raise ValueError("⚠️ PINECONE_API_KEY no está configurado. Añádelo a tu archivo .env")

# Initialize Pinecone
pc = Pinecone(api_key=api_key)
index_name = "rag-handbook"

# Crear index automáticamente si no existe
if index_name not in pc.list_indexes().names():
    print(f"📦 Creando index '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=1024,  # Titan Embed V2
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ Index '{index_name}' creado exitosamente")

# Conectar al vector store
vector_store = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)
print(f"✅ Conectado a Pinecone index: {index_name}")

📦 Creando index 'rag-handbook'...
✅ Index 'rag-handbook' creado exitosamente
✅ Conectado a Pinecone index: rag-handbook


In [6]:
ids = vector_store.add_documents(documents=all_splits)

In [7]:
results = vector_store.similarity_search(
    "How many days of vacation does an employee get in their first year?"
)

display(Markdown(results[0].page_content))

business travel. This includes transportation, lodging, meals, and incidental expenses within
established limits. Receipts must be submitted within 14 days of travel. First-class travel, personal
expenses, and non-business activities are not reimbursable. Employees should exercise good
judgment and cost-effective decision-making when traveling on behalf of the company.

## RAG Agent

In [8]:
from langchain.tools import tool

@tool
def search_handbook(query: str) -> str:
    """Search the employee handbook for information"""
    results = vector_store.similarity_search(query)
    return results[0].page_content

In [9]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
# from langchain_aws import ChatBedrock
# llm = ChatBedrock(
# model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# # model_id="cohere.command-r-plus-v1:0",
# region_name="us-east-1",
# model_kwargs={
# "temperature": 0.5,
# "max_tokens": 2048,
# "top_p": 0.9,
# }
# )


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

llm = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)

# Create the RAG agent
agent = create_agent(
    model=llm,
    tools=[search_handbook]
)

In [10]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many days of vacation does an employee get in their first year?")]}
)

In [11]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='How many days of vacation does an employee get in their first year?', additional_kwargs={}, response_metadata={}, id='fe6f38c3-d845-409e-b368-6860fdcdb317'),
              AIMessage(content=[{'type': 'text', 'text': "<thinking>The user is asking about the number of vacation days an employee gets in their first year. This information is likely to be found in the employee handbook. I should use the 'search_handbook' tool to find this information.</thinking>\n"}, {'type': 'tool_use', 'name': 'search_handbook', 'input': {'query': 'How many days of vacation does an employee get in their first year?'}, 'id': 'tooluse_kP_MMlwYRYiss3MB2xiqHw'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'dd220522-3280-4695-9151-9fd780b335f6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 02 Jan 2026 23:57:39 GMT', 'content-type': 'application/json', 'content-length': '620', 'connection': 'keep-alive', 'x-amzn-requestid': 'dd220522-3280

In [12]:
JSON(response)

<IPython.core.display.JSON object>